In [1]:
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()
# Access the API key
api_key = os.getenv("api_key")
if api_key:
    print(f"API Key ditemukan")
else:
    print("API Key tidak ditemukan cek file .env")

API Key ditemukan


In [14]:
from roboflow import Roboflow
rf = Roboflow(api_key="QO6LigDvwLNNNkRHdCn3")
project = rf.workspace("ellbendls-p1vkz").project("project1-adrzg")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to project1-1 in yolov8:: 100%|██████████| 1422/1422 [00:03<00:00, 369.53it/s]


In [ ]:
import os
import torch
from ultralytics import YOLO
import yaml

# Check GPU dan setup multi-GPU
print("=== GPU INFO ===")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU count: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

# Set environment untuk multi-GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1' if torch.cuda.device_count() >= 2 else '0'

# Konfigurasi dataset
data_config = {
    'path': '/kaggle/working/project1-1',  # Ganti dengan nama dataset Kaggle Anda
    'train': 'train/images',
    'val': 'valid/images',
    'nc': 3,  # 3 class
    'names': ['botol_kaca', 'botol_kaleng', 'botol_plastik']
}

# Simpan konfigurasi
with open('/kaggle/working/trash_data.yaml', 'w') as f:
    yaml.dump(data_config, f)

# Load model YOLOv8
model = YOLO('yolov8n.pt')  # bisa yolov8s.pt untuk akurasi lebih tinggi (lebih berat)

# Determine batch size dan device berdasarkan GPU count
gpu_count = torch.cuda.device_count()
if gpu_count >= 2:
    # Untuk T4x2
    batch_size = 32  # 16 per GPU
    device = [0, 1]  # Multi-GPU
    print("Using T4x2 Multi-GPU training")
else:
    # Untuk P100 single GPU
    batch_size = 24
    device = 0
    print("Using P100 Single GPU training")

# Training dengan multi-GPU support
results = model.train(
    data='/kaggle/working/trash_data.yaml',  # Gunakan config yang baru dibuat
    epochs=100,
    imgsz=640,  # Ganti dari 224 ke 640 (standar YOLO)
    batch=batch_size,  # Batch size yang sudah dioptimasi
    device=device,  # Multi-GPU atau single GPU
    name='trash_detector',
    
    # Optimasi tambahan untuk Kaggle
    amp=True,  # Mixed precision training (penting untuk T4)
    workers=8,  # Dataloader workers
    optimizer='AdamW',  # Lebih baik dari SGD default
    patience=20,  # Early stopping
    save_period=25,  # Save checkpoint setiap 25 epochs
    project='/kaggle/working/runs'  # Save ke working directory
)

# Save best model ke output directory
import shutil
best_model_path = '/kaggle/working/runs/detect/trash_detector/weights/best.pt'
if os.path.exists(best_model_path):
    shutil.copy2(best_model_path, '/kaggle/working/best_trash_detector.pt')
    print("Best model saved to: /kaggle/working/best_trash_detector.pt")

New https://pypi.org/project/ultralytics/8.3.160 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.70  Python-3.10.13 torch-2.1.2 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=project1-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=trash_detector18, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=N

100%|██████████| 755k/755k [00:00<00:00, 4.07MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

...

train: Scanning C:\Files\University\Research\trash_class_robot\project1-1\train\labels... 490 images, 3 backgrounds, 0 corrupt: 100%|██████████| 490/490 [00:00<00:00, 937.46it/s]


train: New cache created: C:\Files\University\Research\trash_class_robot\project1-1\train\labels.cache


val: Scanning C:\Files\University\Research\trash_class_robot\project1-1\valid\labels... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<00:00, 734.48it/s]

val: New cache created: C:\Files\University\Research\trash_class_robot\project1-1\valid\labels.cache


KeyboardInterrupt: 

# Model dilatih di T4x2nya Kaggle

Testing

In [3]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image
import json
from datetime import datetime

def test_model():
    """Test model pada semua gambar di validation folder"""
    
    # Load model yang sudah di-train
    model_path = 'best.pt'
    
    if not os.path.exists(model_path):
        print("❌ Model tidak ditemukan! Pastikan file best.pt ada.")
        return
    
    model = YOLO(model_path)
    print("✅ Model loaded successfully!")
    
    # Path ke validation images
    val_images_path = 'project1-1/valid/images'
    
    if not os.path.exists(val_images_path):
        print(f"❌ Validation images path tidak ditemukan: {val_images_path}")
        return
    
    # Ambil SEMUA gambar di folder (tidak hanya 5)
    all_images = [f for f in os.listdir(val_images_path) 
                  if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    
    if not all_images:
        print("❌ Tidak ada gambar ditemukan di folder validation!")
        return
    
    print(f"🖼️  Ditemukan {len(all_images)} gambar untuk testing...")
    
    # Buat folder untuk save results
    os.makedirs('test_results', exist_ok=True)
    os.makedirs('test_results/annotated_images', exist_ok=True)
    
    # Statistics tracking
    test_stats = {
        'total_images': len(all_images),
        'total_detections': 0,
        'images_with_detections': 0,
        'class_counts': {'botol_kaca': 0, 'botol_kaleng': 0, 'botol_plastik': 0},
        'confidence_threshold': 0.3,
        'timestamp': datetime.now().isoformat(),
        'model_path': model_path,
        'results': []
    }
    
    # Class names dan colors
    class_names = ['botol_kaca', 'botol_kaleng', 'botol_plastik']
    colors = {
        'botol_kaca': (0, 0, 255),      # Merah
        'botol_kaleng': (0, 255, 0),    # Hijau  
        'botol_plastik': (255, 0, 0),   # Biru
    }
    
    # Test setiap gambar
    print("🔄 Memproses semua gambar...")
    
    for i, img_name in enumerate(all_images):
        img_path = os.path.join(val_images_path, img_name)
        
        if i % 10 == 0:  # Progress update setiap 10 gambar
            print(f"   Progress: {i+1}/{len(all_images)} ({(i+1)/len(all_images)*100:.1f}%)")
        
        try:
            # Load image untuk annotasi
            img = cv2.imread(img_path)
            if img is None:
                print(f"   ⚠️  Tidak bisa load gambar: {img_name}")
                continue
            
            # Predict
            results = model(img_path, verbose=False)
            
            # Parse detections
            detections = []
            annotated_img = img.copy()
            
            if results[0].boxes is not None:
                boxes = results[0].boxes
                for box in boxes:
                    confidence = float(box.conf[0])
                    class_id = int(box.cls[0])
                    
                    # Filter berdasarkan confidence threshold
                    if confidence >= test_stats['confidence_threshold']:
                        if class_id < len(class_names):
                            class_name = class_names[class_id]
                            bbox = box.xyxy[0].tolist()  # [x1, y1, x2, y2]
                            
                            detection = {
                                'class': class_name,
                                'confidence': confidence,
                                'bbox': bbox
                            }
                            detections.append(detection)
                            
                            # Update statistics
                            test_stats['class_counts'][class_name] += 1
                            test_stats['total_detections'] += 1
                            
                            # Draw bounding box pada gambar
                            x1, y1, x2, y2 = [int(coord) for coord in bbox]
                            color = colors[class_name]
                            
                            # Bounding box
                            cv2.rectangle(annotated_img, (x1, y1), (x2, y2), color, 2)
                            
                            # Label dengan background
                            label = f"{class_name}: {confidence:.2f}"
                            label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                            cv2.rectangle(annotated_img, (x1, y1-25), (x1 + label_size[0] + 10, y1), color, -1)
                            cv2.putText(annotated_img, label, (x1+5, y1-8),
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            # Update statistics jika ada deteksi
            if detections:
                test_stats['images_with_detections'] += 1
                
                # Save annotated image
                output_path = os.path.join('test_results/annotated_images', f"annotated_{img_name}")
                cv2.imwrite(output_path, annotated_img)
            
            # Save hasil per gambar
            image_result = {
                'image_name': img_name,
                'num_detections': len(detections),
                'detections': detections
            }
            test_stats['results'].append(image_result)
            
        except Exception as e:
            print(f"   ❌ Error processing {img_name}: {e}")
            continue
    
    # Hitung statistik akhir
    avg_detections = test_stats['total_detections'] / test_stats['total_images'] if test_stats['total_images'] > 0 else 0
    detection_rate = test_stats['images_with_detections'] / test_stats['total_images'] * 100 if test_stats['total_images'] > 0 else 0
    
    # Tambahkan ke stats
    test_stats['avg_detections_per_image'] = avg_detections
    test_stats['detection_rate_percent'] = detection_rate
    
    # Save hasil ke JSON
    with open('test_results/test_statistics.json', 'w') as f:
        json.dump(test_stats, f, indent=2)
    
    # Print hasil
    print("\n" + "="*60)
    print("📊 HASIL TESTING LENGKAP")
    print("="*60)
    print(f"📁 Total gambar diproses: {test_stats['total_images']}")
    print(f"🎯 Total deteksi: {test_stats['total_detections']}")
    print(f"📈 Gambar dengan deteksi: {test_stats['images_with_detections']} ({detection_rate:.1f}%)")
    print(f"📊 Rata-rata deteksi per gambar: {avg_detections:.2f}")
    print(f"🔍 Confidence threshold: {test_stats['confidence_threshold']}")
    
    print("\n📋 DISTRIBUSI KELAS:")
    for class_name, count in test_stats['class_counts'].items():
        percentage = count / test_stats['total_detections'] * 100 if test_stats['total_detections'] > 0 else 0
        print(f"   {class_name}: {count} deteksi ({percentage:.1f}%)")
    
    print(f"\n💾 Hasil disimpan ke:")
    print(f"   - test_results/test_statistics.json (statistik lengkap)")
    print(f"   - test_results/annotated_images/ (gambar dengan bounding box)")
    
    # Buat summary visualization jika ada matplotlib
    try:
        create_summary_plots(test_stats)
        print(f"   - test_results/summary_plots.png (grafik statistik)")
    except Exception as e:
        print(f"   ⚠️  Tidak bisa membuat plot: {e}")
    
    print("="*60)
    print("✅ Testing selesai!")
    
    return test_stats

def create_summary_plots(stats):
    """Buat grafik summary dari hasil testing"""
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Testing Summary - Trash Detector Model', fontsize=16, fontweight='bold')
    
    # 1. Detection Rate
    labels = ['With Detections', 'Without Detections']
    sizes = [stats['images_with_detections'], 
             stats['total_images'] - stats['images_with_detections']]
    colors = ['#2ecc71', '#e74c3c']
    
    ax1.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    ax1.set_title('Detection Rate')
    
    # 2. Class Distribution
    classes = list(stats['class_counts'].keys())
    counts = list(stats['class_counts'].values())
    colors_bar = ['#e74c3c', '#2ecc71', '#3498db']
    
    bars = ax2.bar(classes, counts, color=colors_bar)
    ax2.set_title('Class Distribution')
    ax2.set_ylabel('Number of Detections')
    
    # Tambahkan nilai di atas bar
    for bar, count in zip(bars, counts):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{count}', ha='center', va='bottom')
    
    # 3. Detections per Image Histogram
    detections_per_img = [len(result['detections']) for result in stats['results']]
    ax3.hist(detections_per_img, bins=range(max(detections_per_img) + 2), 
             color='#9b59b6', alpha=0.7, edgecolor='black')
    ax3.set_title('Detections per Image Distribution')
    ax3.set_xlabel('Number of Detections')
    ax3.set_ylabel('Number of Images')
    
    # 4. Key Statistics
    ax4.axis('off')
    stats_text = f"""
    KEY STATISTICS
    
    Total Images: {stats['total_images']}
    Total Detections: {stats['total_detections']}
    
    Images with Detections: {stats['images_with_detections']}
    Detection Rate: {stats['detection_rate_percent']:.1f}%
    
    Avg Detections/Image: {stats['avg_detections_per_image']:.2f}
    Confidence Threshold: {stats['confidence_threshold']}
    
    Most Detected Class: {max(stats['class_counts'], key=stats['class_counts'].get)}
    """
    
    ax4.text(0.1, 0.9, stats_text, transform=ax4.transAxes, fontsize=12,
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
    
    plt.tight_layout()
    plt.savefig('test_results/summary_plots.png', dpi=300, bbox_inches='tight')
    plt.close()

if __name__ == "__main__":
    print("=" * 60)
    print("🧪 TRASH DETECTOR - BATCH TESTING")
    print("by DecX-x | 2025-06-30")
    print("=" * 60)
    
    test_model()

🧪 TRASH DETECTOR - BATCH TESTING
by DecX-x | 2025-06-30
✅ Model loaded successfully!
🖼️  Ditemukan 145 gambar untuk testing...
🔄 Memproses semua gambar...
   Progress: 1/145 (0.7%)
   Progress: 11/145 (7.6%)
   Progress: 21/145 (14.5%)
   Progress: 31/145 (21.4%)
   Progress: 41/145 (28.3%)
   Progress: 51/145 (35.2%)
   Progress: 61/145 (42.1%)
   Progress: 71/145 (49.0%)
   Progress: 81/145 (55.9%)
   Progress: 91/145 (62.8%)
   Progress: 101/145 (69.7%)
   Progress: 111/145 (76.6%)
   Progress: 121/145 (83.4%)
   Progress: 131/145 (90.3%)
   Progress: 141/145 (97.2%)

📊 HASIL TESTING LENGKAP
📁 Total gambar diproses: 145
🎯 Total deteksi: 147
📈 Gambar dengan deteksi: 145 (100.0%)
📊 Rata-rata deteksi per gambar: 1.01
🔍 Confidence threshold: 0.3

📋 DISTRIBUSI KELAS:
   botol_kaca: 29 deteksi (19.7%)
   botol_kaleng: 13 deteksi (8.8%)
   botol_plastik: 105 deteksi (71.4%)

💾 Hasil disimpan ke:
   - test_results/test_statistics.json (statistik lengkap)
   - test_results/annotated_images/ 

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import json
from datetime import datetime

class TrashDetector:
    def __init__(self, model_path, confidence_threshold=0.3):
        self.model = YOLO(model_path)
        self.confidence_threshold = confidence_threshold
        self.class_names = ['plastic_bottle', 'can', 'glass_bottle']
        
        # Warna untuk setiap kategori (BGR)
        self.colors = {
            'plastic_bottle': (255, 0, 0),    # Biru
            'can': (0, 255, 0),              # Hijau  
            'glass_bottle': (0, 0, 255),     # Merah
            'other': (128, 128, 128)         # Abu-abu
        }
    
    def detect_and_classify(self, frame):
        """Deteksi objek dan klasifikasi dengan threshold"""
        results = self.model(frame, verbose=False)
        detections = []
        
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    # Koordinat bounding box
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    confidence = box.conf[0].cpu().numpy()
                    class_id = int(box.cls[0].cpu().numpy())
                    
                    # Tentukan kategori berdasarkan confidence
                    if confidence >= self.confidence_threshold:
                        category = self.class_names[class_id]
                    else:
                        category = 'other'
                    
                    # Hitung center point untuk robot
                    center_x = int((x1 + x2) / 2)
                    center_y = int((y1 + y2) / 2)
                    
                    detection = {
                        'category': category,
                        'confidence': float(confidence),
                        'bbox': [int(x1), int(y1), int(x2), int(y2)],
                        'center': [center_x, center_y],
                        'timestamp': datetime.now().isoformat()
                    }
                    detections.append(detection)
        
        return detections
    
    def draw_detections(self, frame, detections):
        """Gambar bounding box dan label pada frame"""
        for det in detections:
            x1, y1, x2, y2 = det['bbox']
            center_x, center_y = det['center']
            category = det['category']
            confidence = det['confidence']
            
            # Pilih warna berdasarkan kategori
            color = self.colors.get(category, (0, 255, 255))
            
            # Gambar bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # Gambar center point
            cv2.circle(frame, (center_x, center_y), 5, color, -1)
            
            # Label dan confidence
            label = f"{category}: {confidence:.2f}"
            cv2.putText(frame, label, (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
            # Koordinat untuk robot
            coord_text = f"({center_x}, {center_y})"
            cv2.putText(frame, coord_text, (center_x-50, center_y+20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
        
        return frame
    
    def save_coordinates_for_robot(self, detections, output_file='robot_coordinates.json'):
        """Simpan koordinat untuk robot"""
        robot_data = {
            'timestamp': datetime.now().isoformat(),
            'detections': []
        }
        
        for det in detections:
            robot_detection = {
                'category': det['category'],
                'center_x': det['center'][0],
                'center_y': det['center'][1],
                'confidence': det['confidence']
            }
            robot_data['detections'].append(robot_detection)
        
        with open(output_file, 'w') as f:
            json.dump(robot_data, f, indent=2)
        
        return robot_data

# Implementasi real-time detection
def main():
    # Inisialisasi detector
    detector = TrashDetector('runs/detect/trash_detector/weights/best.pt')
    
    # Buka kamera
    cap = cv2.VideoCapture(0)  # 0 untuk webcam default
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    print("Tekan 'q' untuk keluar, 's' untuk screenshot")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Deteksi sampah
        detections = detector.detect_and_classify(frame)
        
        # Gambar hasil deteksi
        frame_with_detection = detector.draw_detections(frame.copy(), detections)
        
        # Simpan koordinat untuk robot (jika ada deteksi)
        if detections:
            robot_data = detector.save_coordinates_for_robot(detections)
            print(f"Terdeteksi {len(detections)} objek: {robot_data}")
        
        # Tampilkan frame
        cv2.imshow('Trash Detector', frame_with_detection)
        
        # Kontrol keyboard
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s'):
            cv2.imwrite(f'screenshot_{datetime.now().strftime("%Y%m%d_%H%M%S")}.jpg', 
                       frame_with_detection)
            print("Screenshot disimpan!")
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()